In [1]:
from graph_tool import *
from tqdm import tqdm
import csv

In [2]:
# get dict with REMID as key and corresponding CREM as value
# dict with CREM as key and contained REMs as value
crem_dict = dict()
rem_crem_dict = dict()
with open('../data/clusterREMs_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "REMID":
            crem_dict[row[0]] = [row[1], row[3], row[4]]
            if rem_crem_dict.get(row[1]) == None:
                rem_crem_dict[row[1]] = [row[0]]
            else:
                rem_crem_dict[row[1]].append(row[0])

1264526it [00:04, 258328.93it/s]


In [3]:
# get dict with GENE name as key and start and end as value
gene_dict = dict()
with open('../data/GeneAnnotation_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        if row[0] != "chr":
            gene_dict[row[3]] = [int(row[1]), int(row[2]), row[0]]

58175it [00:00, 456764.57it/s]


In [4]:
sample_celltype_dict = dict()
ID_celltype_dict = dict()
with open('../data/sampleToCelltype.csv') as sampleToCelltype:
    reader = csv.reader(sampleToCelltype)
    for row in reader:
        if row[0] != "sampleID":
            # load dict with sample and corresponding celltype
            sample_celltype_dict[row[0]] = [row[1], row[2]]
            
            # load dict with celltype ID and corresponding celltype
            ID_celltype_dict[row[1]] = row[2]

# print(sample_celltype_dict)
# print(ID_celltype_dict)

In [5]:
# dict for genes with their expression values in each sample for both data sets (Roadmap and Blueprint)
gene_expr_dict = dict()
with open('../data/GeneExpressionRoadmap_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        species = row[3]
        gene = row[0]
        sample = row[1]
        exprs = row[2]
#         if sample not in sample_set:
#             continue
        if species != "species":
            if gene_expr_dict.get(gene) == None:
                gene_expr_dict[gene] = {sample : exprs}
            elif gene_expr_dict[gene].get(sample) == None:
                gene_expr_dict.get(gene)[sample] = exprs
                
with open('../data/GeneExpressionBlueprint_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        species = row[3]
        gene = row[0]
        sample = row[1]
        exprs = row[2]
#         if sample not in sample_set:
#             continue
        if species != "species":
            if gene_expr_dict.get(gene) == None:
                gene_expr_dict[gene] = {sample : exprs}
            elif gene_expr_dict[gene].get(sample) == None:
                gene_expr_dict.get(gene)[sample] = exprs


6399141it [00:09, 707915.01it/s]
3257745it [00:04, 658893.64it/s]


In [18]:
# rem_dict{celltypes : {samples: {REMs: score}}}
# dict to map celltype to samples and the included REMs with their scores

rem_dict = dict()
with open('../data/REMActivity_1.csv') as file_1:
    reader_1 = csv.reader(file_1)
    for r in tqdm(reader_1):
        rem = r[0] 
        sample = r[1] 
        score = r[3] 
        
        if sample_celltype_dict.get(sample) == None:       # if sample not 
            continue
        else:
            celltype = sample_celltype_dict.get(sample)[0]
            
        if rem_dict.get(celltype) == None:
            rem_dict[celltype] = {sample: {rem: score}}
            
        elif rem_dict[celltype].get(sample) == None:
            rem_dict.get(celltype)[sample] = {rem: score}
            
        else:
            rem_dict[celltype].get(sample)[rem] = score
            

399206927it [15:12, 437721.10it/s]


In [6]:
# create template graph to fill with celltype specific values afterwards
# Graph contains gens and rems and their edges buth with no values

g = Graph()
name = g.new_vp("string")
g.vp.name = name
type = g.new_vp("string")
g.vp.type = type
start = g.new_vp("int")
g.vp.start = start
end = g.new_vp("int")
g.vp.end = end
rem = g.new_ep("string")
g.ep.rem = rem
chr = g.new_vp("string")
g.vp.chr = chr
celltypeID = g.new_ep("string")
g.edge_properties["celltypeID"] = celltypeID
celltype = g.new_ep("string")
g.edge_properties["celltype"] = celltype

done_proteins_dict = {}
done_crems = {}
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        rem_name = row[4]
        gen = row[3]
        crem = crem_dict.get(rem_name)
        if crem != None:
            crem = crem[0]
        if row[1] != "start":
            
            if crem in done_crems:
                v1 = g.vertex(done_crems.get(crem))
            elif crem != None:
                v1 = g.add_vertex()
                g.vp.name[v1] = crem
                g.vp.type[v1] = "REM"
                g.vp.start[v1] = crem_dict.get(rem_name)[1]
                g.vp.end[v1] = crem_dict.get(rem_name)[2]
                done_crems[crem] = int(v1)
            else:
                v1 = g.add_vertex()
                g.vp.name[v1] = rem_name
                g.vp.type[v1] = "REM"
                g.vp.start[v1] = int(row[1])
                g.vp.end[v1] = int(row[2])

            if done_proteins_dict.get(gen) == None:
                v2 = g.add_vertex()
                g.vp.name[v2] = gen
                g.vp.type[v2] = "Gene"
                g.vp.start[v2] = gene_dict.get(gen)[0]
                g.vp.end[v2] = gene_dict.get(gen)[1]
                done_proteins_dict[gen] = int(v2)
            else:
                v2 = g.vertex(done_proteins_dict.get(gen))

            e = g.add_edge(v1, v2)
            g.ep.rem[e] = rem_name
                
g.save(f"../data/graph-tool/template.gt")

2404862it [00:59, 40658.98it/s]


In [20]:
# load template graph and fill with celltype edge values by iterating over samples in celltype

graph = load_graph(f"../data/graph-tool/template.gt")

for celltype_id in rem_dict:
    if celltype_id not in ["CTID_0000006"]:
        continue
    print(celltype_id)
    
    g = Graph(graph)
    samples = rem_dict.get(celltype_id)
    celltype_name = ID_celltype_dict.get(celltype_id)
    
    for sample in samples:
        sampleID = g.new_ep("double")
        g.edge_properties[sample] = sampleID
        sample_g = g.new_vp("double")
        g.vertex_properties[sample]= sample_g
    
        for e in tqdm(g.edges()):
            rem = g.vp.name[e.source()]
            g.ep.celltypeID[e] = celltype_id
            g.ep.celltype[e] = celltype_name
            
            gene = g.vp.name[e.target()]                      # new part, adding gene expr for specific sample
            expr = gene_expr_dict.get(gene).get(sample) 
            g.vp[sample][e.target()] = expr
            
#             if rem.startswith("C"):    #  mean val for crems
#                 mean_expr = sum(float(samples.get(sample).get(r)) for r in rem_crem_dict.get(rem))/len(rem_crem_dict.get(rem))
#                 g.ep[sample][e] = mean_expr
#             else:
#                 g.ep[sample][e] = samples.get(sample).get(rem) 
            
            rem_edge_name = g.ep.rem[e]
            g.ep[sample][e] = samples.get(sample).get(rem_edge_name)
            
    g.save(f"../data/graph-tool/{celltype_id}.gt")
#     break

CTID_0000006


2404861it [03:06, 12895.20it/s]
2404861it [03:02, 13162.86it/s]
2404861it [03:02, 13190.65it/s]


In [3]:
g = load_graph("../data/graph-tool/CTID_0000006.gt")

In [6]:
for e in g.edges():
    if g.vp.name[e.target()] == "ENSG00000233750" and g.vp.name[e.source()] == "CREM0000007":
        print(g.ep.B_C0010KB1[e])
        print(g.ep.rem[e])

-0.012413861189268384
REM0000022
0.00012564005500229042
REM0000023


In [11]:
g.list_properties()

B_C0010KB1     (vertex)  (type: double)
B_C0011IB1     (vertex)  (type: double)
B_C001UYB4     (vertex)  (type: double)
end            (vertex)  (type: int32_t)
name           (vertex)  (type: string)
start          (vertex)  (type: int32_t)
type           (vertex)  (type: string)
B_C0010KB1     (edge)    (type: double)
B_C0011IB1     (edge)    (type: double)
B_C001UYB4     (edge)    (type: double)
celltype       (edge)    (type: string)
celltypeID     (edge)    (type: string)
rem            (edge)    (type: string)


In [19]:
counter = 0
for e in g.edges():
    counter += 1
    print(g.ep.B_C0010KB1[e])
    print(g.vp.name[e.source()])
    print(g.vp.name[e.target()])
    if counter == 10:
        break

0.031891038020643886
REM0000001
ENSG00000278267
0.35818993348847716
REM0000002
ENSG00000223972
-0.17473665019991788
CREM0000001
ENSG00000223972
-0.21302234023002894
CREM0000001
ENSG00000237613
-0.1323817964763507
REM0000005
ENSG00000237613
0.05655310368116036
CREM0000002
ENSG00000227232
-0.05825563063323182
CREM0000002
ENSG00000237613
0.1283341595085099
CREM0000003
ENSG00000227232
0.09849975482203463
CREM0000003
ENSG00000237613
-0.3270380201701096
REM0000010
ENSG00000278267


In [ ]:
# check if all edges are the same in all samples
d = dict()
with open('../data/REMActivity_1.csv') as file:
    reader = csv.reader(file)
    counter = 0
    for row in tqdm(reader):
        if row[0] != "REMID":
            if d.get(row[1]) != None:
                d[row[1]].add(row[0])
            else:
                d[row[1]] = {row[0]}
        
check = list(d.values())[0]
for k in d.keys():
#     print(k)
    if  len(check.difference(d.get(k))) != 0:
        print(k)

In [ ]:
for celltype_id in rem_dict:
    print(celltype_id)
    
    g = Graph()
    name = g.new_vp("string")
    g.vp.name = name
    type = g.new_vp("string")
    g.vp.type = type
    celltypeID = g.new_ep("string")
    g.edge_properties["celltypeID"] = celltypeID
    celltype = g.new_ep("string")
    g.edge_properties["celltype"] = celltype
    
#         standDnase1Log2 = g.new_ep("double")
#         g.edge_properties["standDnase1Log2"] = standDnase1Log2

    done_proteins_dict = {}
    with open('../data/REMAnnotationModelScore_1.csv') as file:
        reader = csv.reader(file)
        for row in tqdm(reader):
            rem = row[4]
            gen = row[3]
            if row[1] != "start":
                v1 = g.add_vertex()
                g.vp.name[v1] = rem
                g.vp.type[v1] = "REM"

                if done_proteins_dict.get(gen) == None:
                    v2 = g.add_vertex()
                    g.vp.name[v2] = gen
                    g.vp.type[v2] = "Gene"
                    done_proteins_dict[gen] = int(v2)
                else:
                    v2 = g.vertex(done_proteins_dict.get(gen))

                e = g.add_edge(v1, v2)
#                     g.ep.standDnase1Log2[e] = rem_dict.get(rem)[1]
                g.ep.celltypeID[e] = celltype_id
                g.ep.celltype[e] = ID_celltype_dict.get(celltype_id)
                
                samples = rem_dict.get(celltype_id)
                for sample in samples:
                    sampleID = g.new_ep("double")
                    g.edge_properties[sample] = sampleID
                    g.ep[sample][e] = samples.get(sample).get(rem)
                
    g.save(f"../data/graph-tool/{celltype_id}.gt")


In [3]:
l = []
with open('../data/REMAnnotationModelScore_1.csv') as file:
    reader = csv.reader(file)
    for row in tqdm(reader):
        rem = row[4]
        l.append(rem)

2404862it [00:06, 388620.87it/s]


In [4]:
len(set(l))

2404862